In [31]:
#site i, #participant j, feature g

from spicy import stats
import numpy as np
import pandas as pd

#######
# from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nnv
# import torch.optim as optim
##########
np.random.seed(666)
#Total sample size (sum of all sites) is 5000
N=500
#the number of features is 1000.
G=10
#10 sites
I=10

In [32]:
#This section will be the same for both the homogeneous and heterogeneous versions.
#for homogenous version, we have the same number of samples in each site
n_i=int(N/I)
n_samples=[n_i for _ in range(I)]
###
alpha_G = np.random.uniform(0, 0.5,G)

Y_I=np.random.uniform(0,0.1,I)
tau_I=stats.invgamma.rvs(a=2, scale=0.5,size=I)

df = pd.DataFrame({'Y_i': Y_I, 'tau_i': tau_I})
gamma_IG=df.apply(lambda x: stats.norm.rvs(loc=x['Y_i'], scale=np.sqrt(x['tau_i']), size=G), axis=1)

lambda_I=stats.gamma.rvs(a=50,scale=50,size=I)
v_I=stats.gamma.rvs(a=50,scale=1,size=I)
df1=pd.DataFrame({'lambda_i':lambda_I,'v_i':v_I})
delta_IG=df1.apply(lambda x:stats.gamma.rvs(a=x['lambda_i']*x['v_i'],scale=x['v_i'],size=G),axis=1)

sigma_G=stats.halfcauchy.rvs(scale=0.2, size=G)

#####################################################################################################
#check for finite value
# print("tau_i:", tau_i)
# print("gamma_ig:", gamma_ig)
# print("delta_ig:", delta_ig)

def check_finite_array(arr, name):
    if not np.isfinite(arr).all():
        print(f"Warning: {name} contains inf or NaN!")
    else:
        print(f"{name} is finite.")

def check_finite_series(series, name):
    for idx, row in series.items():
        if not np.isfinite(row).all():
            print(f"Warning: {name}[{idx}] contains inf or NaN!")
    else:
        print(f'{name} is finite')

check_finite_array(tau_I, "tau_i")
check_finite_series(gamma_IG, "gamma_ig")
check_finite_series(delta_IG, "delta_ig")
print('done')
######################################################################################################

#fixed effects of age and sex, different for different features
theta_age=stats.norm.rvs(loc=0,scale=1,size=G)
theta_sex=stats.norm.rvs(loc=0,scale=1,size=G)
theta_G=pd.DataFrame({"age":theta_age,"sex":theta_sex})
# print(theta)

tau_i is finite.
gamma_ig is finite
delta_ig is finite
done


In [56]:
sex=[]#it is a list with 10 entries, where each entry contains generated data for one site
age=[]#it is a list with 10 entries, where each entry contains generated data for one site

for n in n_samples:
    #covariates sex
    #for site i with n_i samples, the probability of male is equal to female as 50%.
    v1=np.random.binomial(n=1, p=0.5, size=n)
    sex.append(v1)
    #covariate age
    #by reading the median of all boxplots, I guess the mean to be 40 and 10 for sd.
    v2=stats.norm.rvs(loc=40, scale=10, size=n)
    v2 = v2[v2 > 0]
    print(len(v2))
    while len(v2) < n:
        additional_samples = stats.norm.rvs(loc=40, scale=10, size=n - len(v2))
        v2 = np.concatenate((v2, additional_samples[additional_samples > 0]))
    age.append(v2)

#need to be standardized within site
standardized_age = [(a - np.mean(a)) / np.std(a) for a in age]

#no need to standardize sex as it is 0 or 1
# print(sex[0].shape)
# print(age[0].shape)

50
50
50
50
50
50
50
50
50
50


In [34]:
#fixed effect transform function
def fixed_effect_linear(x, theta):  # x: 2x1 and theta: 2x1
    return np.dot(x.T, theta)  

In [35]:
#simulate features for all sites when assuming linear for fixed effect
data = []
for i in range(I):#for each site
    J = n_i  # n_i is same for all sites
    gamma_i = gamma_IG[i]
    delta_i = delta_IG[i]    
    x = pd.DataFrame({'age': standardized_age[i], 'sex': sex[i]})  # Ensure age[i] and sex[i] are iterable
    
    data1 = []
    for j in range(J):#for each participant
        each_participant = []  # generated features for each participant
        x_ij = x.iloc[j, :]

        for g in range(G):#for each feature
            sigma_g = sigma_G[g]
            theta_g = theta_G.iloc[g, :]
            epsilon_ijg = stats.norm.rvs(loc=0, scale=sigma_g, size=1)[0]  
            
            phi = fixed_effect_linear(x_ij, theta_g)
            y_ijg = alpha_G[g] + phi + gamma_i[g] + delta_i[g] * epsilon_ijg

            each_participant.append(y_ijg)

        data1.append(each_participant)

    data1 = pd.DataFrame(data1, columns=[f'feature{g}' for g in range(G)])
    data1['site'] = i
    data.append(data1)

final_data = pd.concat(data, ignore_index=True)
print(final_data)
#this simulated feature values are assumed to be value after standardization


         feature0      feature1       feature2      feature3      feature4  \
0   -1.833502e+06 -1.592200e+07   64230.566340 -1.062412e+06  4.847096e+05   
1    6.835585e+04  5.927245e+06  -61939.625548 -4.241112e+06  5.160394e+05   
2   -2.070052e+06  6.381244e+05  133711.407945  6.219031e+05 -1.012306e+05   
3   -1.691034e+05  1.046735e+07  214768.755691 -3.453047e+05  2.413620e+05   
4   -5.179555e+05  4.203051e+06  402667.583860  1.639996e+05  5.704886e+05   
..            ...           ...            ...           ...           ...   
495 -1.295978e+05 -3.727192e+06  -23265.615661 -1.423903e+06  1.049783e+05   
496  1.485251e+05  2.754234e+06 -233875.877219  1.077260e+05  1.265957e+06   
497  3.838380e+05  6.218149e+06 -160242.827068  4.267388e+05  9.803067e+05   
498  5.852227e+05 -2.758260e+06 -386131.235137  3.048058e+06  1.362252e+05   
499  6.219042e+05  2.975229e+06 -360834.381083  3.367052e+06 -6.663439e+05   

         feature5      feature6      feature7      feature8    

In [54]:
class MLP(nn.Module):
    #two input features, 1 hidden layer, one output, use sigmoid to transform the weighted sum of the inputs to non-linear
    def __init__(self, input_dim, hidden_dim, output_dim,classification=False):
        super(MLP, self).__init__()
        #self.layers = nn.ModuleList()
        #prev_dim = input_dim
        
        #for hidden_dim in hidden_dims:
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        nn.init.normal_(self.hidden_layer.weight, mean=0, std=1)  
        nn.init.normal_(self.hidden_layer.bias, mean=0, std=1)    
       
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        nn.init.normal_(self.output_layer.weight, mean=0, std=1)  
        nn.init.normal_(self.output_layer.bias, mean=0, std=1)    
   
    def forward(self, x):
        #for layer in self.layers:
        x = torch.sigmoid(self.hidden_layer(x))  # sigmoid activation function
        return self.output_layer(x) #output without activation function


def fixed_effect_nonlinear(x):#multi-layer perceptron with logistic activation functions
    X_tensor=torch.tensor(x.values,dtype=torch.float32)#comvert datafrmae to numpy
    input_dim=2
    hidden_dim=1
    output_dim=1
    model=MLP(input_dim,hidden_dim,output_dim)
    model.eval()
    y_simulated=model(X_tensor).detach().numpy()
    return(y_simulated)
# print(len(gamma_ig))
#simulate features for all sites when assuming nonlinear for fixed effect
phi_G = []  
standardized_age1 = np.concatenate(standardized_age)
sex1 = np.concatenate(sex)
# print(sex1)
x = pd.DataFrame({'age': standardized_age1, 'sex': sex1})
# print(x.values)

for g in range(G):  
    d = fixed_effect_nonlinear(x)  
    phi_G.append(d)

#convert phi to be in the format of for each site
phi_I=[]
for i in range(I):
    d=[]
    for g in range(G):
        phi_g=phi_G[g]
        a=i*n_i
        b=(i+1)*n_i
        phi_ig=phi_g[a:b]
        d.append(phi_ig)
    phi_I.append(np.column_stack(d))
# print(phi_I[1].shape)   

#print(len(phi[0]))
data = []
for i in range(I):#for each site
    J = n_i  # n_i is same for all sites
    gamma_i = gamma_IG[i]
    delta_i = delta_IG[i]    
    x = pd.DataFrame({'age': standardized_age[i], 'sex': sex[i]})  # Ensure age[i] and sex[i] are iterable
    phi_i=pd.DataFrame(phi_I[i])
    data1 = []
    for j in range(J):#for each participant
        each_participant = []  # generated features for each participant
        x_ij = x.iloc[j, :]

        for g in range(G):#for each feature
            sigma_g = sigma_G[g]
            theta_g = theta_G.iloc[g, :]
            epsilon_ijg = stats.norm.rvs(loc=0, scale=sigma_g, size=1)[0]  
            phi=phi_i.iloc[j,g]            
            y_ijg = alpha_G[g] + phi + gamma_i[g] + delta_i[g] * epsilon_ijg

            each_participant.append(y_ijg)

        data1.append(each_participant)

    data1 = pd.DataFrame(data1, columns=[f'feature{g}' for g in range(G)])
    data1['site'] = i
    data.append(data1)

final_data = pd.concat(data, ignore_index=True)
print(final_data)

         feature0      feature1       feature2      feature3      feature4  \
0    2.564527e+05 -2.385620e+05 -163421.979171 -3.701143e+06  3.347463e+05   
1   -1.980927e+05 -1.955470e+06  293476.945080 -6.426854e+06 -6.588653e+05   
2   -2.910220e+05  5.299499e+06  447974.901065  4.589916e+06  6.107997e+04   
3   -9.756237e+05  8.804577e+06   27943.500108 -7.771699e+05  3.608159e+05   
4    7.375940e+04 -1.084992e+07 -380722.666021  5.721812e+06 -9.071929e+04   
..            ...           ...            ...           ...           ...   
495 -7.117956e+05 -6.505574e+06  331886.743274 -1.378294e+06  7.542409e+05   
496  1.451611e+06 -1.103609e+07  -32936.474682 -4.528141e+06 -3.556612e+05   
497 -1.382723e+06 -1.217991e+06  383093.834699  5.617075e+04 -5.065342e+02   
498  1.000726e+06 -7.433897e+06  422214.251116  4.507125e+06  2.808423e+05   
499  7.876548e+05 -6.842811e+06  287602.181774  4.382123e+06  1.082518e+06   

         feature5      feature6      feature7      feature8    